<a href="https://colab.research.google.com/github/Aryamaan777/Seq2Seq/blob/main/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download de
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import numpy as np
import spacy
import random
from torch.utils.tensorboard import SummaryWriter  # to print to tensorboard
#from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint

In [3]:
spacy_ger = spacy.load("de")
spacy_eng = spacy.load("en")

In [4]:
def tokenize_ger(text):
  return [tok.text for tok in spacy_ger.tokenizer(text)]

def tokenize_eng(text):
  return [tok.text for tok in spacy_eng.tokenizer(text)]

In [5]:
german=Field(tokenize=tokenize_ger, lower=True, init_token="<sos>",eos_token="<eos>")
english=Field(tokenize=tokenize_eng,lower=True, init_token="<sos>",eos_token="<sos>")

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [6]:
train_data,valid_data,test_data=Multi30k.splits(
    exts=(".de",".en"),fields=(german,english)
)

/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [7]:
german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

In [8]:
class Encoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(Encoder, self).__init__()
    self.dropout=nn.Dropout(p)
    self.hidden_size=hidden_size
    self.num_layers=num_layers

    self.embedding=nn.Embedding(input_size, embedding_size)
    self.rnn=nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

  def forward(self,x):
    embedding=self.dropout(self.embedding(x))
    outputs,(hidden,cell)=self.rnn(embedding)
    return hidden,cell

In [9]:
class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p):
    super(Decoder,self).__init__()
    self.dropout=nn.Dropout(p)
    self.hidden_size=hidden_size
    self.num_layers=num_layers

    self.embedding=nn.Embedding(input_size, embedding_size)
    self.rnn=nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
    self.fc=nn.Linear(hidden_size,output_size)

  def forward(self, x, hidden, cell):
    x=x.unsqueeze(0)

    embedding=self.dropout(self.embedding(x))
    outputs,(hidden,cell)=self.rnn(embedding,(hidden,cell))
    predictions=self.fc(outputs)
    predictions=predictions.squeeze(0)

    return predictions, hidden, cell

In [10]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder):
    super(Seq2Seq, self).__init__()
    self.encoder=encoder
    self.decoder=decoder

  def forward(self, source, target, teacher_force_ratio=0.5):
    batch_size = source.shape[1]
    target_len = target.shape[0]
    target_vocab_size = len(english.vocab)

    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    hidden, cell = self.encoder(source)

    x=target[0]

    for t in range(1,target_len):
      output, hidden, cell=self.decoder(x, hidden, cell)

      outputs[t]=output

      best_guess=output.argmax(1)

      x=target[t] if random.random()<teacher_force_ratio else best_guess
    
    return outputs

In [11]:
num_epochs=100
learning_rate=0.001
batch_size=64

In [23]:
load_model=False
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size_encoder=len(german.vocab)
input_size_decoder=len(english.vocab)
output_size=len(english.vocab)
encoder_embedding_size=300
decoder_embedding_size=300
hidden_size=1024
num_layers=2
enc_dropout=0.5
dec_dropout=0.5

In [13]:
step=0

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data,valid_data,test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device
)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [14]:
encoder_net=Encoder(input_size_encoder, encoder_embedding_size, hidden_size, num_layers, enc_dropout).to(device)

decoder_net=Decoder(input_size_decoder, decoder_embedding_size, hidden_size, output_size, num_layers, dec_dropout).to(device)

In [24]:
model=Seq2Seq(encoder_net, decoder_net).to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

##CHECK THIS BEFORE RUNNING
load_model=True

In [19]:
sentence = "ein boot mit mehreren männern darauf wird von einem großen pferdegespann ans ufer gezogen."

In [25]:
torch.cuda.is_available()

False

In [19]:
for epoch in range(num_epochs):
  print(f"[Epoch {epoch} / {num_epochs}]")

  checkpoint = {"state_dict":model.state_dict(), "optimizer": optimizer.state_dict()}
  # save_checkpoint(checkpoint)

  model.eval()

  # translated_sentence = translate_sentence(
  #     model, sentence,german, english, device, max_length=50
  # )

  # print(f"Translated Example Sentence: \n {translated_sentence}")

  model.train()

  for batch_idx, batch in enumerate(train_iterator):
    inp_data=batch.src.to(device)
    target=batch.trg.to(device)

    output=model(inp_data, target)

    output=output[1:].reshape(-1, output.shape[2])
    target=target[1:].reshape(-1)

    optimizer.zero_grad()
    loss=criterion(output,target)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    optimizer.step()
    step+=1

[Epoch 0 / 100]
[Epoch 1 / 100]
[Epoch 2 / 100]
[Epoch 3 / 100]
[Epoch 4 / 100]
[Epoch 5 / 100]
[Epoch 6 / 100]
[Epoch 7 / 100]
[Epoch 8 / 100]
[Epoch 9 / 100]
[Epoch 10 / 100]
[Epoch 11 / 100]
[Epoch 12 / 100]
[Epoch 13 / 100]
[Epoch 14 / 100]
[Epoch 15 / 100]
[Epoch 16 / 100]
[Epoch 17 / 100]
[Epoch 18 / 100]
[Epoch 19 / 100]
[Epoch 20 / 100]
[Epoch 21 / 100]
[Epoch 22 / 100]
[Epoch 23 / 100]
[Epoch 24 / 100]
[Epoch 25 / 100]
[Epoch 26 / 100]
[Epoch 27 / 100]
[Epoch 28 / 100]
[Epoch 29 / 100]
[Epoch 30 / 100]
[Epoch 31 / 100]
[Epoch 32 / 100]
[Epoch 33 / 100]
[Epoch 34 / 100]
[Epoch 35 / 100]
[Epoch 36 / 100]
[Epoch 37 / 100]
[Epoch 38 / 100]
[Epoch 39 / 100]
[Epoch 40 / 100]
[Epoch 41 / 100]
[Epoch 42 / 100]
[Epoch 43 / 100]
[Epoch 44 / 100]
[Epoch 45 / 100]
[Epoch 46 / 100]
[Epoch 47 / 100]
[Epoch 48 / 100]
[Epoch 49 / 100]
[Epoch 50 / 100]
[Epoch 51 / 100]
[Epoch 52 / 100]
[Epoch 53 / 100]
[Epoch 54 / 100]
[Epoch 55 / 100]
[Epoch 56 / 100]
[Epoch 57 / 100]
[Epoch 58 / 100]
[Epoch 

In [20]:
!pip install --upgrade torchtext

     |████████████████████████████████| 7.0MB 7.2MB/s 
     |████████████████████████████████| 776.8MB 24kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [27]:
from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint

In [22]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

In [23]:
checkpoint = {"state_dict":model.state_dict(), "optimizer": optimizer.state_dict()}
save_checkpoint(checkpoint)

=> Saving checkpoint


In [32]:
if load_model:
  load_checkpoint(torch.load("my_checkpoint.pth.tar",map_location=torch.device('cpu')), model, optimizer)

=> Loading checkpoint


In [35]:
score=bleu(test_data, model, german, english, device)
print(f"Bleu Score: {score*100:.2f}")

Bleu Score: 5.38
